<a href="https://colab.research.google.com/github/mbibrahe/BA_Project/blob/main/Copy_of_UNET%2B%2B_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
!git clone https://github.com/lim-eren/CDnet2014.git

fatal: destination path 'CDnet2014' already exists and is not an empty directory.


In [139]:
#!ls */train/*

In [140]:
from __future__ import print_function
import glob, os, numpy as np
from keras.preprocessing import image as kImage
from __future__ import print_function
import numpy as np
import random as rn
import os
from tensorflow.compat.v1.keras import backend as K




In [141]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [142]:

"""download = drive.CreateFile({'id': '1MUOVUdNCDTHA0FuHo4IsONhxSLLMxdZ7'})
download.GetContentFile("a2_n.zip")"""
"""
download = drive.CreateFile({'id': '1v--STVVyw32UYZO4ouINWP3HnciRwkIZ'})
download.GetContentFile("t_train.zip")


#! unzip a2_n.zip
! unzip t_train.zip"""

'\ndownload = drive.CreateFile({\'id\': \'1v--STVVyw32UYZO4ouINWP3HnciRwkIZ\'})\ndownload.GetContentFile("t_train.zip")\n\n\n#! unzip a2_n.zip\n! unzip t_train.zip'

In [143]:
download = drive.CreateFile({'id': '1WwOiIWXIjdA7Ng_dE7uD4Qi8sQ1MUkG-'})
download.GetContentFile("Full_test_1.zip")


#! unzip a2_n.zip
! unzip Full_test_1.zip

Streaming output truncated to the last 5000 lines.
  inflating: Full_test_1/train/x/7/in002100.jpg  
  inflating: Full_test_1/train/x/7/in002101.jpg  
  inflating: Full_test_1/train/x/7/in002102.jpg  
  inflating: Full_test_1/train/x/7/in002103.jpg  
  inflating: Full_test_1/train/x/7/in002104.jpg  
  inflating: Full_test_1/train/x/7/in002105.jpg  
  inflating: Full_test_1/train/x/7/in002106.jpg  
  inflating: Full_test_1/train/x/7/in002107.jpg  
  inflating: Full_test_1/train/x/7/in002108.jpg  
  inflating: Full_test_1/train/x/7/in002109.jpg  
  inflating: Full_test_1/train/x/7/in002110.jpg  
  inflating: Full_test_1/train/x/7/in002111.jpg  
  inflating: Full_test_1/train/x/7/in002112.jpg  
  inflating: Full_test_1/train/x/7/in002113.jpg  
  inflating: Full_test_1/train/x/7/in002114.jpg  
  inflating: Full_test_1/train/x/7/in002115.jpg  
  inflating: Full_test_1/train/x/7/in002116.jpg  
  inflating: Full_test_1/train/x/7/in002117.jpg  
  inflating: Full_test_1/train/x/7/in002118.jpg  

In [144]:
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling3D
from keras.models import Sequential
from keras.layers.convolutional import Deconv2D
from keras.layers import Activation, Input, Dropout, BatchNormalization, SpatialDropout2D, LayerNormalization, UpSampling2D, Conv2DTranspose
from keras.layers.merge import concatenate, Average
import keras.backend as K
import tensorflow as tf
from keras import regularizers
from keras.regularizers import l2

smooth = 1.
dropout_rate = 0.5

def standard_unit(input_tensor, stage, nb_filter, kernel_size=3):

    act = 'relu'

    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, kernel_initializer = 'he_normal', padding='same')(input_tensor)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, kernel_initializer = 'he_normal', padding='same')(x)
    x = Dropout(dropout_rate)(x)

    return x

def unet_mod(input1_, input2_):
  con1_ = concatenate([input1_, input2_], axis=-1)
  nb_filter = [32,64,128,256,512]
  """conv1_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_)
  conv1_1 = Dropout(0.5)(conv1_1)
  conv1_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_1)"""
  conv1_1 = standard_unit(con1_, stage='11', nb_filter=nb_filter[0])
  pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_1)
  
  """conv2_1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  #conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
  conv2_1 = Dropout(0.5)(conv2_1)
  conv2_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_1)"""
  conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
  pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_1)

  up1_2 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2_1)
  conv1_2 = concatenate([up1_2, conv1_1], axis=3)
  conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])
  #conv1_2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_2)
  #conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])

  #conv3_1 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  """conv3_1 = Dropout(0.5)(conv3_1)
  conv3_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_1)"""
  #conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
  conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
  pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_1)

  up2_2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv3_1)
  conv2_2 = concatenate([up2_2, conv2_1], axis=3)
  conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])
  #conv2_2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_2)
  #conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])

  up1_3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2_2)
  conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis=3)
  conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])
  #conv1_3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_3)
  #conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])

  """conv4_1 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4_1 = Dropout(0.5)(conv4_1)
  conv4_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_1)"""
  #conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
  conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
  pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_1)

  up3_2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4_1)
  conv3_2 = concatenate([up3_2, conv3_1], axis=3)
  conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=nb_filter[2])
  #conv3_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_2)
  #conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=256)

  up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), padding='same')(conv3_2)
  conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis=3)
  conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])
  #conv2_3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_3)
  #conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])

  up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_3)
  conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], axis=3)
  conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])
  #conv1_4 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_4)
  #conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])

  """conv5_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5_1 = Dropout(0.5)(conv5_1)
  conv5_1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5_1)"""
  conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])
  #conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])

  up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), padding='same')(conv5_1)
  conv4_2 = concatenate([up4_2, conv4_1], axis=3)
  conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])
  #conv4_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_2)
  #conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])

  up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), padding='same')(conv4_2)
  conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis=3)
  conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])
  #conv3_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3_3)
  #conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])

  up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), padding='same')(conv3_3)
  conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis=3)
  conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])
  #conv2_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2_4)
  #conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])

  up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), padding='same')(conv2_4)
  conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis=3)
  conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])
  #conv1_5 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1_5)
  #conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])
  #nestnet_output_4 = Conv2D(2, 3, activation='relu', kernel_initializer = 'he_normal', padding='same')(conv1_5)
  #nestnet_output_4 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_5)


  #Deep supervision
  """nestnet_output_1 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same')(conv1_2)
  nestnet_output_2 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same')(conv1_3)
  nestnet_output_3 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same')(conv1_4)
  nestnet_output_4 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same')(conv1_5)"""

  nestnet_output_1 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_2)
  nestnet_output_2 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_3)
  nestnet_output_3 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_4)
  nestnet_output_4 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_5)


  #con_ = concatenate([nestnet_output_1, nestnet_output_2, nestnet_output_3, nestnet_output_4], axis=3)
  #con_ = Average((nestnet_output_1, nestnet_output_2, nestnet_output_3, nestnet_output_4))
  con_ = tf.keras.layers.Average()([nestnet_output_1, nestnet_output_2, nestnet_output_3, nestnet_output_4])
  



  return con_



In [148]:
import numpy as np

import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling3D
from keras.models import Sequential
from keras.layers.convolutional import Deconv2D
from keras.layers import Activation, Input, Dropout, BatchNormalization, SpatialDropout2D, LayerNormalization, UpSampling2D, Conv2DTranspose
from keras.layers.merge import concatenate
import keras.backend as K
import tensorflow as tf
from keras import regularizers
from PIL import Image as PILimage
import matplotlib.pyplot as plt
from matplotlib import image as imageMat
from keras import backend as K
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as Image
import cv2 as cv
#import torch

#tf.random.set_seed(10)

#Use tensorflow 2.0!!
#Softmax
#Dont save in .JPG!
#Stay with .png or numpy array

# load data
dataset_path = os.path.join('Full_test_1')
#dataset_path = os.path.join('Full_test_1_40samples')
def getData(dataset_dir):
    X_list1= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','1','*.jpg')))
    X_list2= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','2','*.jpg')))
    X_list3= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','3','*.jpg')))
    X_list4= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','4','*.jpg')))
    X_list5= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','5','*.jpg')))
    X_list6= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','6','*.jpg')))
    X_list7= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','7','*.jpg')))
    X_list8= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','8','*.jpg')))
    X_list9= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'x','9','*.jpg')))

    #Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
    Y_list1= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','1','*.png')))
    Y_list2= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','2','*.png')))
    Y_list3= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','3','*.png')))
    Y_list4= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','4','*.png')))
    Y_list5= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','5','*.png')))
    Y_list6= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','6','*.png')))
    Y_list7= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','7','*.png')))
    Y_list8= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','8','*.png')))
    Y_list9= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'y','9','*.png')))

    Z_list1= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','1','*.jpg')))
    Z_list2= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','2','*.jpg')))
    Z_list3= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','3','*.jpg')))
    Z_list4= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','4','*.jpg')))
    Z_list5= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','5','*.jpg')))
    Z_list6= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','6','*.jpg')))
    Z_list7= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','7','*.jpg')))
    Z_list8= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','8','*.jpg')))
    Z_list9= sorted(glob.glob(os.path.join(dataset_dir, 'train', 'z','9','*.jpg')))
    
    Z_list = sorted(glob.glob(os.path.join(dataset_dir, 'z' ,'*.jpg')))
    
    X= []
    Y= []
    Z= []

    #Run for less frames
    for i in range(len(X_list1)):
        # Load input image
        x1 = cv.resize(cv.imread(X_list1[i]), (320, 240))
        normalizedImg1 = np.zeros((320, 240))
        normalizedImg1 = cv.normalize(x1,  normalizedImg1, 0, 255, cv.NORM_MINMAX)

        x2 = cv.resize(cv.imread(X_list2[i]), (320, 240))
        normalizedImg2 = np.zeros((320, 240))
        normalizedImg2 = cv.normalize(x2,  normalizedImg2, 0, 255, cv.NORM_MINMAX)
        

        x3 = cv.resize(cv.imread(X_list3[i]), (320, 240))
        normalizedImg3 = np.zeros((320, 240))
        normalizedImg3 = cv.normalize(x3,  normalizedImg3, 0, 255, cv.NORM_MINMAX)
        

        x4 = cv.resize(cv.imread(X_list4[i]), (320, 240))
        normalizedImg4 = np.zeros((320, 240))
        normalizedImg4 = cv.normalize(x4,  normalizedImg4, 0, 255, cv.NORM_MINMAX)
        

        x5 = cv.resize(cv.imread(X_list5[i]), (320, 240))
        normalizedImg5 = np.zeros((320, 240))
        normalizedImg5 = cv.normalize(x5,  normalizedImg5, 0, 255, cv.NORM_MINMAX)
        

        x6 = cv.resize(cv.imread(X_list6[i]), (320, 240))
        normalizedImg6 = np.zeros((320, 240))
        normalizedImg6 = cv.normalize(x6,  normalizedImg6, 0, 255, cv.NORM_MINMAX)
        

        x7 = cv.resize(cv.imread(X_list7[i]), (320, 240))
        normalizedImg7 = np.zeros((320, 240))
        normalizedImg7 = cv.normalize(x7,  normalizedImg7, 0, 255, cv.NORM_MINMAX)
        

        x8 = cv.resize(cv.imread(X_list8[i]), (320, 240))
        normalizedImg8 = np.zeros((320, 240))
        normalizedImg8 = cv.normalize(x8,  normalizedImg8, 0, 255, cv.NORM_MINMAX)
        

        x9 = cv.resize(cv.imread(X_list9[i]), (320, 240))
        normalizedImg9 = np.zeros((320, 240))
        normalizedImg9 = cv.normalize(x9,  normalizedImg9, 0, 255, cv.NORM_MINMAX)
        
        X.append(normalizedImg1)
        X.append(normalizedImg2)
        X.append(normalizedImg3)
        X.append(normalizedImg4)
        #X.append(normalizedImg5)
        #X.append(normalizedImg6)
        X.append(normalizedImg7)
        X.append(normalizedImg8)
        X.append(normalizedImg9)

        # Load ground-truth in grayscale and normalized
        y1 = cv.resize(cv.imread(Y_list1[i], 0), (320, 240))
        y1 = cv.normalize(y1, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y2 = cv.resize(cv.imread(Y_list2[i], 0), (320, 240))
        y2 = cv.normalize(y2, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y3 = cv.resize(cv.imread(Y_list3[i], 0), (320, 240))
        y3 = cv.normalize(y3, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y4 = cv.resize(cv.imread(Y_list4[i], 0), (320, 240))
        y4 = cv.normalize(y4, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y5 = cv.resize(cv.imread(Y_list5[i], 0), (320, 240))
        y5 = cv.normalize(y5, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y6 = cv.resize(cv.imread(Y_list6[i], 0), (320, 240))
        y6 = cv.normalize(y6, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y7 = cv.resize(cv.imread(Y_list7[i], 0), (320, 240))
        y7 = cv.normalize(y7, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y8 = cv.resize(cv.imread(Y_list8[i], 0), (320, 240))
        y8 = cv.normalize(y8, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

        y9 = cv.resize(cv.imread(Y_list9[i], 0), (320, 240))
        y9 = cv.normalize(y9, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        
        Y.append(y1)
        Y.append(y2)
        Y.append(y3)
        Y.append(y4)
        #Y.append(y5)
        #Y.append(y6)
        Y.append(y7)
        Y.append(y8)
        Y.append(y9)

        # Load background image
        """z = cv.resize(cv.imread(Z_list[i]), (320, 240))
        #z = cv.normalize(z, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalizedImg = np.zeros((320, 240))
        normalizedImg = cv.normalize(z,  normalizedImg, 0, 255, cv.NORM_MINMAX)
        Z.append(normalizedImg)"""

        z1 = cv.resize(cv.imread(Z_list1[i]), (320, 240))
        normalizedImg1 = np.zeros((320, 240))
        normalizedImg1 = cv.normalize(z1,  normalizedImg1, 0, 255, cv.NORM_MINMAX)

        z2 = cv.resize(cv.imread(Z_list2[i]), (320, 240))
        normalizedImg2 = np.zeros((320, 240))
        normalizedImg2 = cv.normalize(z2,  normalizedImg2, 0, 255, cv.NORM_MINMAX)
        

        z3 = cv.resize(cv.imread(Z_list3[i]), (320, 240))
        normalizedImg3 = np.zeros((320, 240))
        normalizedImg3 = cv.normalize(z3,  normalizedImg3, 0, 255, cv.NORM_MINMAX)
        

        z4 = cv.resize(cv.imread(Z_list4[i]), (320, 240))
        normalizedImg4 = np.zeros((320, 240))
        normalizedImg4 = cv.normalize(z4,  normalizedImg4, 0, 255, cv.NORM_MINMAX)
        

        z5 = cv.resize(cv.imread(Z_list5[i]), (320, 240))
        normalizedImg5 = np.zeros((320, 240))
        normalizedImg5 = cv.normalize(z5,  normalizedImg5, 0, 255, cv.NORM_MINMAX)
        

        z6 = cv.resize(cv.imread(Z_list6[i]), (320, 240))
        normalizedImg6 = np.zeros((320, 240))
        normalizedImg6 = cv.normalize(z6,  normalizedImg6, 0, 255, cv.NORM_MINMAX)
        

        z7 = cv.resize(cv.imread(Z_list7[i]), (320, 240))
        normalizedImg7 = np.zeros((320, 240))
        normalizedImg7 = cv.normalize(z7,  normalizedImg7, 0, 255, cv.NORM_MINMAX)
        

        z8 = cv.resize(cv.imread(Z_list8[i]), (320, 240))
        normalizedImg8 = np.zeros((320, 240))
        normalizedImg8 = cv.normalize(z8,  normalizedImg8, 0, 255, cv.NORM_MINMAX)
        

        z9 = cv.resize(cv.imread(Z_list9[i]), (320, 240))
        normalizedImg9 = np.zeros((320, 240))
        normalizedImg9 = cv.normalize(z9,  normalizedImg9, 0, 255, cv.NORM_MINMAX)
        
        Z.append(normalizedImg1)
        Z.append(normalizedImg2)
        Z.append(normalizedImg3)
        Z.append(normalizedImg4)
        #Z.append(normalizedImg5)
        #Z.append(normalizedImg6)
        Z.append(normalizedImg7)
        Z.append(normalizedImg8)
        Z.append(normalizedImg9)
        #Z.append(z)
    
        
            
    X = np.asarray(X)
    Y = np.asarray(Y)
    Z = np.asarray(Z)
    
    # Shuffle the training data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    Z = Z[idx]

    #måske Z her
    return X, Y, Z

from sklearn import metrics
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.cast(y_pred > t, tf.int32)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1. - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

X, Y, Z = getData(dataset_path)

print("Shape of Y:" + str(Y.shape))
print("Shape of X:" + str(X.shape))
"""for i in range(0,100):
  plt.imshow(X[i])
  plt.show()
  plt.imshow(Y[i])
  plt.show()
  plt.imshow(Z[i])
  plt.show()"""

# init the model

# Testing a U-NET++ (The name comes from the shape of the network, 
# where we first contract, and the expand)
def initModel():
    
    
    img_1 = Input(shape=(None, None ,3))
    img_2 = Input(shape=(None, None ,3))
    



    
    #x = concatenate([img_1, img_2], axis = -1)
    #x = Conv2D(1, (3,3), activation='relu', padding='same')(img_1)
    
    # We concatenate the input image, and the background image
    out1 = unet_mod(img_1, img_2)
    #out2 = unet_mod(img_2)
   
    #x = tf.keras.layers.Average()([out1, out2])
    #x = concatenate([out1, out2], axis = -1)
    #x = Dense(2)(x)


    #out_d = Dense(2)(x)

    #Try dense with , kernel_regularizer=l2(1e-4)
    #out_d = Dense(1, activation='sigmoid', kernel_regularizer=l2(1e-4))(x)
    #out_d = Activation('sigmoid')(out_d)


    
    
    
    model = Model([img_1, img_2], outputs=out1)
    model.summary()
    #Loss functions, other usable loss functions?
    #model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.RMSprop(lr=5e-4), metrics=['accuracy']) optimizer='adam'


    """Computes the cross-entropy loss between true labels and predicted labels.
    Use this cross-entropy loss when there are only two label classes (assumed to be 0 and 1). 
    For each example, there should be a single floating-point value per prediction."""
    #model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr = 3e-5), metrics=['accuracy'])

    """lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-6,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)"""

    model.compile(loss=bce_dice_loss, optimizer='Adam', metrics=["binary_crossentropy", dice_coef])
    #model.compile(loss=keras.losses.binary_crossentropy, optimizer='Adam', metrics=['binary_accuracy'])

    #What to try:
    "img /=255. "
    "Remove activation functions from last set of conv layers"
    "use loss function: Adam(lr = 3e-5) --- THIS WORKED WELL"
    """I changed the number of epochs and steps per epoch, 
    I increased the number of epochs to around 2000 and reduced steps per epoch to be (number of data items/ batch size) as read on another site. 
    I also found new data to use so increased my data pool to 100 images which dramatically improved my results. 
    I also reduced the original network to 4 layers instead of 5 layers. However, it now also works with 5 layers. 
    I am now using niftynet from UCL/KCL for medical imaging analysis and will be changing the learning rate for better results"""
    
    
    return model


    


model = initModel()


patches_bi = []
patches_in = []
patches_bt = []


# Stacking the background image and input images, for every frame

patch_size = (32, 32)

for i in range(len(X)):
    data_in = Image.extract_patches_2d(X[i], patch_size, max_patches=40, random_state=1)
    data_bt = Image.extract_patches_2d(Y[i], patch_size, max_patches=40, random_state=1)
    data_bi = Image.extract_patches_2d(Z[i], patch_size, max_patches=40, random_state=1)
    patches_in.append(data_in)
    patches_bt.append(data_bt)
    patches_bi.append(data_bi)




patches_in = np.concatenate(patches_in)
patches_bt = np.concatenate(patches_bt)
patches_bi = np.concatenate(patches_bi)


print(patches_in.shape)







#1. Complete fitting first

#We need to train on more pictures

# Training should be stopped if error < 3
model.fit([patches_in, patches_bi], y=patches_bt, batch_size=140, epochs=10, verbose=1, validation_split=0.2, shuffle=True)


model.save('my_model.h5')




Shape of Y:(1400, 240, 320)
Shape of X:(1400, 240, 320, 3)
Model: "model_28"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_65 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
input_66 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
concatenate_544 (Concatenate)   (None, None, None, 6 0           input_65[0][0]                   
                                                                 input_66[0][0]                   
__________________________________________________________________________________________________
conv2d_1740 (Conv2D)            

In [ ]:
"""download = drive.CreateFile({'id': '1MUOVUdNCDTHA0FuHo4IsONhxSLLMxdZ7'})
download.GetContentFile("a2_n")
! unzip a2_n.zip"""

#tf.compat.v1.disable_eager_execution()
dataset_path = os.path.join('Full_test_1', 'test')
def getData_test(dataset_dir):
    X_list= sorted(glob.glob(os.path.join(dataset_dir, 'x','*.jpg')))
    Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
    
    
    X= []
    Y= []
    
    for i in range(len(X_list)):
        # Load input image
        """x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        x /= 255.0
        x = np.floor(x)"""
        x = cv.resize(cv.imread(X_list[i]), (320, 240))
        normalizedImg = np.zeros((320, 240))
        normalizedImg = cv.normalize(x,  normalizedImg, 0, 255, cv.NORM_MINMAX)
        #x = cv.normalize(x, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        
        X.append(normalizedImg)
        
        # Load ground-truth label and encode to value 0 and 1
        """x = kImage.load_img(Y_list[i], grayscale = True)
        x = kImage.img_to_array(x)
        x /= 255.0
        x = np.floor(x)"""

        y = cv.resize(cv.imread(Y_list[i], 0), (320, 240))
        y = cv.normalize(y, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        Y.append(y)
            
    X = np.asarray(X)
    Y = np.asarray(Y)
    #Z = np.asarray(Z)
    
    # Shuffle the training data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    #måske Z her
    return X, Y
X, Y = getData_test(dataset_path)

"""for i in range(0,5):
  plt.imshow(X[i])
  plt.show()
  plt.imshow(Y[i])
  plt.show()"""

#merged_array = np.uint8(np.concatenate([X, X], 3))
#2. Prediction 
pred = model.predict([X,X], verbose=1, batch_size=1)




plt.rcParams['figure.figsize'] = (12.0, 9.0)

idx = 1 # image index in a range of [0,9]

img = np.empty(3, dtype=object)
img[0] = X[idx]
img[1] = Y[idx].reshape(Y[idx].shape[0],Y[idx].shape[1])

img[2] = pred[idx].reshape(pred[idx].shape[0],pred[idx].shape[1])
"""t_img = pred[idx].reshape(pred[idx].shape[0],pred[idx].shape[1])
m, n = t_img
   
# Develop Averaging filter(3, 3) mask 
mask = np.ones([3, 3], dtype = int) 
mask = mask / 9
   
# Convolve the 3X3 mask over the image  
img_new = np.zeros([m, n]) 
  
for i in range(1, m-1): 
    for j in range(1, n-1): 
        temp = t_img[i-1, j-1]*mask[0, 0]+t_img[i-1, j]*mask[0, 1]+t_img[i-1, j + 1]*mask[0, 2]+t_img[i, j-1]*mask[1, 0]+ t_img[i, j]*mask[1, 1]+t_img[i, j + 1]*mask[1, 2]+t_img[i + 1, j-1]*mask[2, 0]+t_img[i + 1, j]*mask[2, 1]+t_img[i + 1, j + 1]*mask[2, 2] 
         
        img_new[i, j]= temp 
          
img_new = img_new.astype(np.uint8) 
#cv2.imwrite('blurred.tif', img_new) 

img[3] = img_new
"""

"""def dice_coef(y_true, y_pred, smooth=1):
  y_true = np.expand_dims(y_true, axis=2)
  print("Shape of y_TRUE" + str(y_true.shape))
  print("Shape of y_PRED" + str(y_pred.shape))
  intersection = K.sum(y_true * y_pred, axis=[0,1,2])
  union = K.sum(y_true, axis=[0,1,2]) + K.sum(y_pred, axis=[0,1,2])
  dice = K.mean((2. * intersection + smooth)/(union + smooth))
  return dice"""

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

print(dice_coef(Y[idx], pred[idx]))

title = ['input', 'ground-truth', 'result']
for i in range(3):
  plt.subplot(1, 4, i+1)
  if i==0:
    plt.imshow(img[i].astype('uint8'))
  else:
    plt.imshow(img[i], cmap='gray')
  
  plt.axis('off')
  plt.title(title[i])
plt.show()



